# Link ML tutorial
- [linkml definitions](https://linkml.io/linkml-model/latest/)
- [tutorial](https://linkml.io/linkml/intro/install.html)

## Converting to json schema
- Converting from yaml to json schema

In [1]:
!gen-json-schema personinfo.yaml

{
    "$defs": {
        "Person": {
            "additionalProperties": false,
            "description": "",
            "properties": {
                "age": {
                    "type": [
                        "string",
                        "null"
                    ]
                },
                "aliases": {
                    "type": [
                        "string",
                        "null"
                    ]
                },
                "full_name": {
                    "type": [
                        "string",
                        "null"
                    ]
                },
                "id": {
                    "type": [
                        "string",
                        "null"
                    ]
                },
                "phone": {
                    "type": [
                        "string",
                        "null"
                    ]
                }
            },
            "title": "Person",


## Validating a data file
- note data file is in yaml format
- what if data is in json format?

In [2]:
!linkml-validate -s personinfo.yaml data/data.yaml

No issues found


In [3]:
!linkml-validate -s personinfo.yaml data/bad-data.yaml

[ERROR] [data/bad-data.yaml/0] Additional properties are not allowed ('made_up_field' was unexpected) in /


In [4]:
!linkml-validate -s personinfo.yaml data/bad-data.json

[ERROR] [data/bad-data.json/0] 32 is not of type 'string', 'null' in /age


## Nesting lists of objects
- we create a class called a Container. This Container is defined as the tree root, and has a slot called persons, which collects all the instances of the person class / entity
- You can also add metadata to the container object, e.g the description and name of the dataset

In [5]:
!less personinfo_container.yaml

id: https://w3id.org/linkml/examples/personinfo
name: personinfo
prefixes:
  linkml: https://w3id.org/linkml/
imports:
  - linkml:types
default_range: string

classes:
  Person:
    attributes:
      id:
      full_name:
      aliases:
      phone:
      age:
  Container:
    tree_root: true
    attributes:
      persons:
        multivalued: true
        inlined_as_list: true
        range: Person
personinfo_container.yaml (END)id: https://w3id.org/linkml/examples/personinfo
name: personinfo
prefixes:
  linkml: https://w3id.org/linkml/
imports:
  - linkml:types
default_range: string

classes:
  Person:
    attributes:
      id:
      full_name:
      aliases:
      phone:
      age:
  Container:
    tree_root: true
    attributes:
      persons:
        multivalued: true
        inlined_as_list: true
        range: Person
(END)

lets try validate this new schema with a container as root

In [6]:
!linkml-validate -s personinfo_container.yaml data/data_2.yaml

No issues found


## Visualising the schema

first we get the REST url, just paste the output into your webbrowser

In [7]:
!gen-yuml -f yuml personinfo_container.yaml

https://yuml.me/diagram/nofunky;dir:TB/class/[Container]++- persons 0..*>[Person|id:string %3F;full_name:string %3F;aliases:string %3F;phone:string %3F;age:string %3F],[Container]

alternatively you can save to png

In [11]:
!gen-yuml -f png -d . personinfo_container.yaml

![](Personinfo.png)

Alternatively you can use mermaid

In [9]:
!gen-erdiagram personinfo_container.yaml

```mermaid
erDiagram
Container {

}
Person {
    string id  
    string full_name  
    string aliases  
    string phone  
    string age  
}

Container ||--}o Person : "persons"

```



---

## Adding constraints and performing validation
- Refer to the following for slot definitions [linkml slot definitions](https://linkml.io/linkml-model/latest/docs/specification/03schemas/#slotdefinition-metaclass)

In [1]:
!less schema/personinfo_constain.yaml

id: https://w3id.org/linkml/examples/personinfo
name: personinfo
prefixes:
  linkml: https://w3id.org/linkml/
imports:
  - linkml:types
default_range: string

classes:
  Person:
    attributes:
      id:
        identifier: true     ## unique key for a person
      full_name:
        required: true       ## must be supplied
        description:
          name of the person
      aliases:
        multivalued: true    ## range is a list
        description:
          other names for the person
      phone:
        pattern: "^[\\d\\(\\)\\-]+$"   ## regular expression
schema/personinfo_constain.yamlid: https://w3id.org/linkml/examples/personinfo
name: personinfo
prefixes:
  linkml: https://w3id.org/linkml/
imports:
  - linkml:types
default_range: string

classes:
  Person:
    attributes:
      id:
        identifier: true     ## unique key for a person
      full_name:
        required: true       ## must be supplied
        description:
          name of the person
      aliases:
      

testing another batch of bad data on this new schema with more detailed constraints

In [5]:
!linkml-validate -s schema/personinfo_constrain.yaml data/bad-data_2.yaml

[ERROR] [data/bad-data_2.yaml/0] '1-800-kryptonite' does not match '^[\\d\\(\\)\\-]+$' in /persons/0/phone
[ERROR] [data/bad-data_2.yaml/0] 'full_name' is a required property in /persons/1


You can also just use json schema to validate as well

In [6]:
!gen-json-schema schema/personinfo_constrain.yaml > schema/personinfo_constrain.json

In [11]:
!jsonschema -i data/bad-data_2.json schema/personinfo_constrain.json

/Users/harrijh/projects/link-ml-practice/.venv/bin/jsonschema:5: DeprecationWarning: The jsonschema CLI is deprecated and will be removed in a future version. Please use check-jsonschema instead, which can be installed from https://pypi.org/project/check-jsonschema/
  from jsonschema.cli import main


I think its perferable to just use linkml's validator

## Converting to excel
- [linkml to excel](https://linkml.io/linkml/generators/excel.html)

In [13]:
!gen-excel schema/personinfo_constrain.yaml --output schema/personinfo_constrain.xlsx

Converting from excel/google sheet to yaml

In [ ]:
!sheets2linkml your_excel_file.xlsx -o output_schema.yaml